 ## Deep Learning Meets Hyperspectral Image Analysis

#### ABSTRACT
For our deep learning project, we present the recreation of one algorithm from the paper “Deep-learning-based regression model and hyperspectral imaging for rapid detection of nitrogen concentration in oilseed rape” (Brassica napus L.) leaf. https://www.sciencedirect.com/science/article/pii/S0169743917306780#ec-research-data

We will present:
- First, a recreation of SAE-FNN model.
- Next, a recreation of SAE-FNN hyperparameters optimization. 
- Finally, a predication of nitrogen concentration compared to results in the paper. 

In order to gain additional computing resources, we extended the experiments to Amazon Web Services Elastic Compute Cloud (EC2).

An in-depth discussion of our recreation and additional experiments.



#### MODEL ARCHITECTURE

The model has two phases:

##### 1. SAE
- The sequential model is a linear stack of layers.
- We created a sequential model by calling the keras_model_sequential() function and then a series of layer functions.
- Keras objects are modified particularly in place, therefore it is not necessary for the model to be assigned back after the layers have been are added.
- Therefore, we can claim that SAE represent unsupervised learning. The learning is from the added layers. 
- The model needs to know what input shape (input_shape ) it should expect. For this reason, the first layer in a sequential model (and only the first, because following layers can do automatic shape inference) needs to receive information about its input shape..
- Before training a model, we need to configure the learning process, which is done via the compile() function, that included three arguments:
    1. An optimizer. 
    2. A loss function.
    3. A list of metrics.
    

Notation as mentioned in the paper:

- Input and output layer of $d$ units (=input_shape).
- Hidden layer of $h$ units.
- Activation function- f() (=ADAM, MSE and relu/elu)
- First the algorithm maps the input $ x\in R^{d}$ to the hidden layer and produces the latent activity $ y\in R^{h}$ which is called "encoding".
- Then $y$ is mapped back to an output layer that has the same size of the input layer, which is calles "decoding".
- The reconstructed values are denoted as $ z\in R^{d}$.
- These to stepd can be formulated as:

$ y=f(w_{y}x+b_{y}),$
$ z=f(w_{z}y+b_{z})$
   
  
- Where $w_{y}$ is the input-to-hidden weight matrix.
- $w_{z}$ is the hidden-to-output weight matrix.
- $b_{y}$ and $b_{z}$ denote the bias of hidden and output units.
- The goal of learning is to minimize the "error" between input and output, which are denoted as: $argmin_{W,b_{y},b_{z}}[c(x,z)]$ where $ w_{y}={w}'_{z}=W$

##### 2. FNN
- We used SAE to extract deep spectral features from hyperspectral image, and then these features were used as input data for FNN to predict N concentration.
- Therefore, we can claim that FNN represent supervised learning. 
- Keras models are trained on R matrices or higher dimensional arrays of input data and labels. For training a model, we will use the fit() function.
- For FNN no activation function is used for the output layer of FNN network because it is a regression problem.
- Output of FNN is a single unit denoted as (Notation as mentioned in the paper): 
 $\hat{y}=w_{l}y_{l}+b_{l}$
- $\hat{y}$ is the output numerical value.
- $w_{l}$ is the weight matrix.
- $b_{l}$ is the bias of output unit.
- $y_{l}$ is the outputsof the last encoding layer.


In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image
img=mpimg.imread('2.jpg')
plt.imshow(img)
im = Image.open("2.jpg")
im.rotate(45).show()
plt.axis('off')
plt.show(im)

As we can see in the fig above:

$1, x_{1},x_{2},...,x_{m}$ - Inputs layers.

$w_{0},w_{1},...,w_{m}$- Weight matrix.

Activation function- Relu/elu.


####  INTRODUCTION
The authors of the paper present several algorithms that generate predictions for nitrogen concentration in oilseed rape leaf: PLS, LS-SVM, SAE- FNN (with different units of last layer). The SAE-FNN algorithm works by first training and tuning a neural network using specific hyperparameters and fine tuning, evaluated over a test dataset. In the paper, the authors test the algorithm against several other algorithms, including PLS and LS-SVM, and compare the performance of with these algorithms across multiple different SAE-FNN models which depends on last layer values. In this report, we examine and attempt to recreate several of the results given in the paper. We focus on one particular result, that SEA-FNN10 achieved better results than those yielded by PLS, LS-SVM and other SEA-FNN models.

### OVERVIEW

#### RECREATION GOALS
Our recreate of SAE- FNN10 targets following objectives. Firstly, to recreate the results of the paper, in which the authors found that SAE-FNN10 outperformed other models. Secondary, we hope to learn more about the results. Finally, we want to learn about using EC2.


####  RECREATION SETUPS

In order to recreate the paper results and extend our knowledge, the recreation setup used a Jupyter notebook based EC2 Instance type t2.xlarge with Ubuntu environment.

Our recreation is based on code provided by the authors of the paper, which can be found at https://www.sciencedirect.com/science/article/abs/pii/S0169743917306780#ec-research-data. Notably, that authors did not provide any code to optimization and tuning. Therefore, we wrote scripts in order to optimize and tune SAE-FNN based on the information provided in the paper. As a result, we had to modify the code to allow us faithfully reproduce the results in the paper. Our changes to the original code are below. 

#### DATA DESCRIPTION 
For each sample-nitrogen concentration, spectral data in the range of 380–1030 nm wavelength with 512 bands of each pixel were taken. As mentioned before, we modified the base code and the data in order to perform hyperparameter tuning. Therefore, we combined validation and train set into one data set. The next step was to split the train (80%) and test sets (20%). Tuning parameters performed with Talos package, therefore, we didn't use validation set.

### Workflow
1. Import libraries.
2. Data loading and splitting.
3. Data visualization.
4. Keras sequential Model defintion.
5. Hyperparameters optimization.    
6. Prediction.
7. Summary and conclusion.




 #### IMPORT LIBRARIES

Core libraries for data processing 

In [ ]:
import numpy as np
from numpy.core.umath_tests import inner1d
import pandas as pd
from pandas.core import datetools
import gzip
import time

Visualization libraries

In [ ]:
import matplotlib.pyplot as plt
import six.moves.cPickle as pickle
import matplotlib.ticker as ticker
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from PIL import Image

Statistics tools

In [ ]:
from sklearn.model_selection import train_test_split
import sys
from sklearn.metrics import mean_squared_error, r2_score
from numpy import sqrt
import random
random.seed(100) # for reproducibility

Deep learning tools

In [ ]:
# Keras items
from keras.optimizers import Adam, Nadam
from keras.activations import relu, elu
from keras.losses import binary_crossentropy
from keras.layers import Input, Dense,Dropout
from keras.models import Model, Sequential, save_model, load_model
from keras.callbacks import CSVLogger,ModelCheckpoint
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model

Hyperparameters tuning

In [ ]:

import talos as ta
from talos.metrics.keras_metrics import fmeasure_acc
from talos import live
from talos.utils.gpu_utils import multi_gpu
from talos.utils.gpu_utils import force_cpu
sys.path.insert(0, '/home/ubuntu/talos/')
import wrangle as wr


####  DATA LOADING
For each sample-nitrogen concentration, spectral data in the range of 380–1030 nm wavelength with 512 bands of each pixel were taken. As we mention before we modified the base code and the data in order to perform hyperparameter tuning. Therefore, we combined validation and train set into one data set. The next step was to split to train (80%) and test (20%) sets. Tuning parameters preformed with Talos package, as a result, we didn't use validation set.


In [ ]:
train_dataset="/home/ubuntu/SAE-FNN_Code_Data/data/train_pixels/ramdonpixel_train.pkl.gz"   #train dataset 
validation_dataset="/home/ubuntu/SAE-FNN_Code_Data/data/train_pixels/ramdonpixel_val.pkl.gz"#validation dataset        

In [ ]:
with gzip.open(train_dataset, 'rb') as f:
            train_set1, valid_set1 = pickle.load(f)
with gzip.open(validation_dataset, 'rb') as f:
            train_set2, valid_set2= pickle.load(f)
X=np.concatenate((train_set1[0],train_set2[0],valid_set1[0],valid_set2[0]))# wavelength
y=np.concatenate((train_set1[1],train_set2[1],valid_set1[1],valid_set2[1]))# N concetration
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100, shuffle=True )

#### DATA VISUALIZATION

In [ ]:
data = pd.read_csv("/home/ubuntu/SAE-FNN_Code_Data/data/Meanspectra.csv") #Mean spectra for each sample 
y_mean=data.iloc[:, 2]
X_mean=data.iloc[:, 3:]
band=range(512)
# Plot spectra
plt.figure(figsize=(15,10))
with plt.style.context(('ggplot')):
    plt.plot(band, X_mean.T)
    plt.xlabel('Band number')
    plt.ylabel('Reflectance')    
plt.show()

 Notice that region of bands 1-120, have moderate change while in the region of bands 250-512 have drastic change of reflectance.
 In addition, in region 250- 300 there are significant difference between each band.



####  KERAS SEQUENTIAL MODEL 



In [ ]:
# input: train (70%) and validation (30%) data with grid of paramaters (p)
# output: compiled model and model's data
def model_opti(x_train, y_train, x_val, y_val, params):
    
    #model format
    model = Sequential()
    
    #input layer
    model.add(Dense(params['first_neuron'],
                    input_dim=x_train.shape[1],
                    #this line is used to convey the fact that we wish to tune the activation 
                    #function parameter and find the best fit among 
                    activation=params['activation'],
                    #this line is used to convey the fact that we wish to tune the 
                    #kernel initializer parameter and find the best fit among 
                    kernel_initializer=params['kernel_initializer']))  
    #adding more layers
    #this line is used to convey the fact that we wish to tune the value of the keep probability of 
    #Dropout and find the best fit among the range of real numbers between 0 and 1.(=0)
    model.add(Dropout(params['dropout']))
    model.add(Dense(1, activation=params['last_activation'],
                    kernel_initializer=params['kernel_initializer']))
    
    #compiling the model
    #we declare the loss function and the optimizer
    model.compile(loss=params['losses'],
                  optimizer=params['optimizer'](),
                  metrics=['acc', fmeasure_acc])
    
    #fitting the model on training data
    history = model.fit(x_train, y_train, 
                        validation_data=[x_val, y_val],
                        batch_size=params['batch_size'],
                        epochs=params['epochs'],
                        verbose=1)


    return history, model

#### HYPERPARAMETERS OPTIMIZATION

##### Tuning parameters:

- Hidden_layers: Layer between input and output layers, where artificial neurons take in a set of weighted inputs and produce an output through an activation function.
- Batch_size: Number of patterns appear on the network before the weight matrix is updated. If the batch size is small, patterns would be less repetitive and hence the weights would be all over the place, making convergence difficult. If batch size is high, learning would become slow as only after many iterations will the batch size change.
- Epochs: The number of epochs is the number of times the entire training data is shown to the model. It plays an important role in how well the model fits on the train data. High number of epochs may overfit to the data and may have generalization problems on the test and validation set, also they could cause vanishing and exploding gradient problems.



- Dropout: The keep-probability of the Dropout layer.
- Kernel initializer: Doesn’t play a very significant role, but still preferred to use normal/uniform initialization while using ReLUs/eLUs.
- Optimizer: It is the algorithm used by the model to update weights of every layer after each iteration. 
- Losses: We seek to minimize the error. 


In order to achieve better parameters, we changed the grid range a couple of times.

Assumption that was used: the more epochs the better, until reaching overfitting. Hence, we used a small value of epochs in the tuning section, and in the final model we used a larger value.


In [ ]:
# set the parameter space
p =  {'first_neuron':[7,8,9,10,20],
     'hidden_layers':[3,4,5,10],
     'batch_size': [128],
     'epochs': [20],
     'dropout': [0.5,0.6,0.7,0.8],
     'kernel_initializer': ['normal','uniform'],
     'optimizer': [ Adam],
     'losses': ['mean_squared_error'],
     'activation':[elu,relu],
     'last_activation': ['elu','relu']
}

##### Talos

As we presented earlier, when designing the architecture for an artificial neural network, there exist a variety of parameters that can be tuned.

Just like we have GridSearchCV for hyperparameter optimization within scikit-learn models like Decision Trees, Talos can be applied on Keras models. Talos works similarly to GridSearchCV, by testing all possible combinations of those parameters we have introduced, and chooses the best model, based on what parameter we have asked it to either optimize or reduce (MSE in our case).

Optimize or reduce performed by cross validation K-fold method. As mentioned earlier, that's the reason why we didn't split the data to validation set and just used training set.



In [ ]:
#input: train set, model settings, hyperparamaters grid
#output: csv file with all combinations.
start = time.time()
force_cpu()
t = ta.Scan(x=X_train,
            y=y_train,
            model=model_opti,
            params=p,
            dataset_name='talos tutorial',
            experiment_no='1',
            reduction_metric="val_loss", 
            reduce_loss=True)

end = time.time()
during=end - start


Training time is:

In [ ]:
print(str(during/3600)+" hours")

The next step is to use the Reporting command to evaluate the experiments. Reporting saves a CSV-file where each experiment is stored with its results (talos tutorial_1.csv). In this file, you can see the rounds_epochs, val_loss, val_accuracy, train loss, test accuracy, activation function, and number of neurons for first and second hidden layer, the optimizer and loss function, batch size, and epochs. 

In [ ]:
r = ta.Reporting(t)
#accessing the results data frame
print(r.data.head(5))


By printing the best_params for val_loss, those experiments with the lowest loss value are visualized.
In our case, the combination of the below: 


- First Neurons: 9 
- Number of hidden layers: 3
- Batch size: 128 
- Epochs: We used a small number in order to get fast tune of the model. in the next section we will use larger number (11000).
- Dropout: 0.6
- Kernel initializer: normal
- Optimizer: Adam 
- Loss function: MSE 
- Activation function : elu 
- Last activation function : relu 


 


In [ ]:
# get the best paramaters
print(r.best_params()[0])

#####   Selected model training

Using the best hyperparameters with larger epoch number, in order to fine tune the model

In [ ]:
# set the parameter space
p2 =  {'first_neuron':[9],
     'hidden_layers':[3],
     'batch_size': [128],
     'epochs': [11000],
     'dropout': [0.6],
     'kernel_initializer': ['normal'],
     'optimizer': [ Adam],
     'losses': ['mean_squared_error'],
     'activation':[elu],
     'last_activation': ['relu']
}

In [ ]:
#input: train set, model settings, hyperparamaters grid
#output: csv file with final combination.
start1 = time.time()
force_cpu()
t2 = ta.Scan(x=X_train,
            y=y_train,
            model=model_opti,
            params=p2,
            dataset_name='talos tutorial',
            experiment_no='pred')
end1 = time.time()
during1=end1 - start1


Fine tuning time is:

In [ ]:
print(str(during1/3600)+" hours")

#### PREDICTION
We are ready now to Deploy the model.
Finally, we used the SAE-FNN model in order to predict the test set.
Deploy command prepares a zip-file that can be transferred to another environment or system. The zip files provide us the information about the experiment in “SAE1_results.csv”, the weights of the model, saved as “SAE1_model.h5, and the model in a json format: “SAE1_model.json”.

In [ ]:
ta.Deploy(t2, 'SAE1')
SAE1 = ta.Restore('SAE1.zip')

After we have used the Deploy command, we can get access to the model and use it for Prediction.

In [ ]:
pred=SAE1.model.predict(X_test)

Predication results

In [ ]:
rmse_val = sqrt(mean_squared_error(y_test, pred))     #RMSEP
r2_val = r2_score(y_test, pred)     #R2P
RPDp=np.std(y_test)/rmse_val*1.0    #RPDp

print(" prediction r2 is: " + str(r2_val))
print(" prediction rms error is: " + str(rmse_val))
print(" prediction RPDp is: " + str(RPDp))

####  SUMMARY AND CONCLUSION

The results of this study demonstrate that deep learning model was feasible to be used as Hyperspectral imaging data analysis method for N prediction in oilseed rape leaf.

In addition, Talos is a useful package for solving complex neural network models and deciding on the right combination of parameters. The best model can be generated by running the code one time, instead of after each change of a single parameter. This saves us time and makes it easier to find the best combinations with the lowest loss values.

The results presented above may seem different. However, we still believe that SAE-FNN10 outperforms other algorithms on the types of data trained on, which is the primary conclusion made by the authors of the original paper. We believe that our recreation/extension shows non-similar results due to difference in random state and hyperparameters optimization method.

#### PERFORMANCE MEASURES


Quality of predictions given by the following measurements: 

$ RMSE(y,\hat{y})=\sqrt{\frac{1}{n}\sum_{i=0}^{n-1}(y_{i}-\hat{y}_{i})^2} $ - A measure of the difference.

$ R^2=(y,\hat{y})=1-\frac{\sum_{i=1}^{n}(y_{i}-\hat{y_{i}})^2}{\sum_{i=1}^{n}(y_{i}-\bar{y_{i}})^2} $ -Explained variance.

$ RPDP=\frac{S(y)}{RMSE} $ -Residual predictive deviations.
 
#### The results (paper results):
$ R^2 = 0.678 $ (Paper result: 0.903)

$ Rmse = 0.039 $ (Paper result: 0.307)

$ RPDp = 1.764 $ (Paper result: 3.238)
